# TD8 

## Partie 1

In [1]:
import pandas as pd
import csv
import re
from Corpus import Corpus
from Document import Document

### 1.1

In [2]:
fichier = "discours_US.csv"
data = pd.read_csv("discours_US.csv", sep="\t")
print(data.head())
print(data.columns)


   speaker                                               text            date  \
0  CLINTON  : I'm getting ready for a lot of things, a lot...  April 12, 2015   
1  CLINTON  [ ] : I'll be graduating in May, and on gradua...  April 14, 2015   
2  CLINTON  : Well, thank you all so much for inviting me ...  April 20, 2015   
3  CLINTON  Thank you so much. I am absolutely delighted t...  April 29, 2015   
4  CLINTON  Oh, hello. Hi, how are you? Well, it's wonderf...     May 5, 2015   

                                               descr  \
0   Video Remarks Announcing Candidacy for President   
1  Remarks in a Question and Answer Session at Ki...   
2                    Remarks in Keene, New Hampshire   
3  Address to the David N. Dinkins Leadership & P...   
4  Remarks at a Roundtable with Young Nevada Resi...   

                                                link  
0  http://www.presidency.ucsb.edu/ws/index.php?pi...  
1  http://www.presidency.ucsb.edu/ws/index.php?pi...  
2  http://w

In [3]:
data.value_counts()

speaker  text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [4]:
corpus_discours = Corpus("TD8")
x = 0

for _, row in data.iterrows():

    titre = row["descr"]  
    auteur = row["speaker"]
    date = row["date"]
    url = row["link"]
    texte = str(row["text"])
    
    
    phrases = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', texte) #Eviter abréviations, nom (ex : U.S, Dr. Maboule..)
    
    for phrase in phrases:
        x = x + 1
        doc = Document(     #Création d'un document pour chaque phrase
            titre=titre,
            auteur=auteur,
            date=date,
            url=url,
            texte=phrase
        )

        if x < 10000:
            corpus_discours.add(doc)
        else:
            break
        

print(corpus_discours)
print(len(corpus_discours.id2doc))

Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia University in New York City (April 29, 2015) - CLINTON
Address to the David N. Dinkins Leadership & Public Policy Forum at Columbia

In [5]:
res = corpus_discours.concorde("the")
res2 = corpus_discours.search("thank")
print(res)
print(res2)

                     contexte gauche motif trouvé  \
0     sident. Americans have fought           the   
1                      c times, but           the   
2                     r of those at           the   
3     ica is strong. So I'm hitting           the   
4     I'll be heading to Annapolis,           the   
...                              ...          ...   
8173  again. I want to also address           the   
8174  ss our unyielding support for           the   
8175                              n           the   
8176  ople, amazing people. We wish           the   
8177                      resolving           the   

                      contexte droit  
0     ir way back from tough economi  
1      deck is still stacked in favo  
2      top. Everyday Americans need   
3      road to earn your vote, becau  
4      naval academy, to major in sy  
...                              ...  
8173   Islamic terror attack in Fran  
8174   people of France, and we mour  
8175  ir los

## Partie 2

In [6]:
from SearchEngine import SearchEngine 
import sys
from tqdm.notebook import tqdm


moteur = SearchEngine(corpus_discours)
requete = input("Recherchez...")
n = 10
resultats = moteur.search(requete, n)
print("Recherche : ", requete)
print(resultats)

Calcul des similarités:   0%|          | 0/9999 [00:00<?, ?document/s]

Recherche :  the
                                               Titre   Auteur  \
0  Remarks at a Town Hall Meeting in Portsmouth, ...  CLINTON   
1  Remarks Announcing Candidacy for President in ...    TRUMP   
2  Remarks Announcing Candidacy for President in ...    TRUMP   
3  Remarks at the New School for Social Research ...  CLINTON   
4  Remarks and a Question and Answer Session at t...  CLINTON   
5  Opening Statement to House Select Committee on...  CLINTON   
6             Interview with Chris Matthews of MSNBC  CLINTON   
7  Interview with Alisyn Camerota of CNN's \New D...  CLINTON   
8  Remarks at the CNN Democratic Presidential Tow...  CLINTON   
9  Remarks at the CNN Democratic Presidential Tow...  CLINTON   

                Date  Score de Similarité  
0  December 29, 2015             0.707107  
1      June 16, 2015             0.577350  
2      June 16, 2015             0.577350  
3      July 13, 2015             0.577350  
4  September 9, 2015             0.577350  
5  

### 2.3

## Partie 3

In [7]:
import ipywidgets as widgets
from IPython.display import display

# Widgets
label = widgets.Label("Moteur de recherche")

text = widgets.Text(
    value='', 
    placeholder='Entrez vos mots-clés (mot1 mot2 mot3)', 
    description='Mots clés :',
    style={'description_width': 'initial'}
)

intSlider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Nombre d\'articles à extraire :',
    style={'description_width': 'initial'}
)

search_button = widgets.Button(
    description='Rechercher',
    button_style='primary'
)


output = widgets.Output()

#Fonction déclenchée par le clic du bouton
def clique_bouton(b):
    with output:  #Redirige la sortie vers l'objet Output
        output.clear_output()  #Efface l'affichage précédent
        
        #Récupération des valeurs des widgets
        mots_cles = text.value
        nombre_articles = intSlider.value
        
        #Affichage des informations de recherche
        print(f"Recherche en cours pour : {mots_cles}")
        print(f"Nombre d'articles à extraire : {nombre_articles}")
        
        #Appel de la fonction de recherche
        resultats = moteur.search(mots_cles, nombre_articles)
        
        #Affichage des résultats
        display(resultats)


#Associer la fonction au clic du bouton
search_button.on_click(clique_bouton)

#Interface complète
interface = widgets.VBox([
    label, 
    text, 
    intSlider, 
    search_button, 
    output
])

#Affichage de l'interface
display(interface)

